# Load modules

In [ ]:
#|default_exp models.gcnn

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

In [ ]:
#|hide
from nbdev.showdoc import show_doc

In [ ]:
#|export
import torch
from torch import nn
from fastai.data.core import DataLoader, DataLoaders, Datasets
from fastcore.foundation import *
from fastcore.basics import *
from fastai.torch_core import Module
from molgraph.layers import *
from fastai.layers import SigmoidRange


In [ ]:
#|export
class GCNMPNmodel(Module):
    
    """
    
    Prototype Graph convolutional layer as described by Kipf & Welling (https://arxiv.org/pdf/1609.02907.pdf) and refactored as MPNN formalism.

       
    """
    
    def __init__(self, n_node_features:int, hidden_units:int=100, num_layers:int=2, output_units:int=1, dropout:float=0.15, y_range=None):
        
        """
        
        Arguments:
        
        n_node_features : int
            Number of features of each atom
            
        hidden_units : int
            Number of hidden units in `self.gcn`
            
            
        num_layers : int
            Number of layers
                    
        output_units : int
            Number of output units
                     
        dropout : float (default = 0.15)
            Amount of dropout regularization
        
        y_range : Tuple[float, float]
            A tuple with the min,max interval of the target variable for regression tasks.
            
            
        """
        self.message_function = MessageGCN(n_node_features, hidden_units, num_layers, hidden_units, dropout)
        
        self.update_function = UpdateGCN(hidden_units, hidden_units, num_layers, hidden_units, dropout)
        
        self.readout_function = nn.Linear(hidden_units, output_units)
        
        
        self.sigmoid = SigmoidRange(*y_range) if y_range else None
        
    
    def forward(self, x):
        node_features, edge_features, adjacency_matrix, degree_matrix = x # Get input features 
        message = self.message_function((node_features, adjacency_matrix, degree_matrix))
        node_state = self.update_function(message[0])
        out = self.sigmoid(self.readout_function(node_state))
        return out

In [ ]:
show_doc(GCNMPNmodel)

---

### GCNMPNmodel

>      GCNMPNmodel (n_node_features:int, hidden_units:int=100, num_layers:int=2,
>                   output_units:int=1, dropout:float=0.15, y_range=None)

Prototype Graph convolutional layer as described by Kipf & Welling (https://arxiv.org/pdf/1609.02907.pdf) and refactored as MPNN formalism.

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()